In [152]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [153]:
Train=pd.read_csv("../Dataset/optimal_fertilizer_train.csv")
Test=pd.read_csv("../Dataset/optimal_fertilizer_test.csv")

In [154]:
Train.head()

,id,Temparature,Humidity,Moisture,Soil Type,Crop Type,Nitrogen,Potassium,Phosphorous,Fertilizer Name
0,0,37,70,36,Clayey,Sugarcane,36,4,5,28-28
1,1,27,69,65,Sandy,Millets,30,6,18,28-28
2,2,29,63,32,Sandy,Millets,24,12,16,17-17-17
3,3,35,62,54,Sandy,Barley,39,12,4,10-26-26
4,4,35,58,43,Red,Paddy,37,2,16,DAP


In [155]:
Train["Fertilizer Name"].value_counts()

Fertilizer Name
14-35-14    114436
10-26-26    113887
17-17-17    112453
28-28       111158
20-20       110889
DAP          94860
Urea         92317
Name: count, dtype: int64

In [156]:
X=Train.drop(columns=["id","Fertilizer Name"])
y=Train["Fertilizer Name"]

In [157]:
cat_cols=X.select_dtypes(include="object").columns.tolist()
cat_cols

['Soil Type', 'Crop Type']

In [158]:
int_cols=X.select_dtypes(include="int64").columns.tolist()
int_cols

['Temparature', 'Humidity', 'Moisture', 'Nitrogen', 'Potassium', 'Phosphorous']

In [159]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler,OneHotEncoder,LabelEncoder
from sklearn.compose import ColumnTransformer

label=LabelEncoder()

y=label.fit_transform(y)

scaler=StandardScaler()
onehot=OneHotEncoder()


numaric=Pipeline(
    [
        ("scaler",scaler),
    ]
)

catagorical=Pipeline(
    [
        ("onehot",onehot),
    ]
)

transformer=ColumnTransformer(
    [
        ("standard",numaric,int_cols),
        ("onehot",catagorical,cat_cols),
    ]
)

In [160]:
y=y.ravel()
y

array([4, 4, 2, ..., 0, 3, 6])

In [161]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=42,test_size=.25)

In [174]:
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)


X_train shape: (562500, 22)
X_test shape: (187500, 22)
y_train shape: (562500,)


In [162]:
X_train=transformer.fit_transform(X_train)
X_test=transformer.transform(X_test)

In [171]:
X_train

array([[-0.1257415 , -0.60673137,  1.50992339, ...,  0.        ,
         1.        ,  0.        ],
       [-0.37413952, -0.0049013 , -0.60989574, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.61945254, -1.659934  , -0.01634639, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [-0.37413952, -1.659934  ,  0.4924102 , ...,  0.        ,
         0.        ,  0.        ],
       [ 1.61304459, -1.20856144,  0.23803191, ...,  0.        ,
         0.        ,  0.        ],
       [-0.37413952, -0.75718889,  0.66199574, ...,  0.        ,
         0.        ,  0.        ]])

In [163]:
model=tf.keras.Sequential(
    [
        tf.keras.layers.InputLayer(input_shape=(X_train.shape[1],)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.2),

        tf.keras.layers.Dense(240, activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.2),


        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(7, activation='softmax')
    ]
)

c:\Users\Lenovo\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


In [164]:
model.summary()

Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_28 (Dense)                │ (None, 128)            │         2,944 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_14          │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_14 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_29 (Dense)                │ (None, 240)            │        30,960 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_15          │ (None, 240)            │           960 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_15 (Dropout)            │ (None, 240)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_30 (Dense)                │ (None, 64)             │        15,424 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_31 (Dense)                │ (None, 7)              │           455 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 51,255 (200.21 KB)

 Trainable params: 50,519 (197.34 KB)

 Non-trainable params: 736 (2.88 KB)

In [165]:
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("Model input shape:", model.output_shape)


X_train shape: (562500, 22)
X_test shape: (187500, 22)
Model input shape: (None, 7)


In [166]:
lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='accuracy', factor=0.3, patience=10, min_lr=1e-6
)
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='accuracy', patience=25, restore_best_weights=True
)

checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath='keras/ann_best_model.keras',      
    monitor='accuracy',                 
    mode='max',                        
    save_best_only=True,                 
    verbose=1
)


In [167]:
model.compile(optimizer=tf.keras.optimizers.Adam(.01),loss="sparse_categorical_crossentropy",metrics=['accuracy', 'AUC', 'Precision'])

In [169]:
if hasattr(X_train, 'toarray'):
    X_train = X_train.toarray()
    X_test = X_test.toarray()

In [170]:
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=200,
    batch_size=16,
    callbacks=[early_stopping, lr_scheduler, checkpoint]
)


Epoch 1/200


InvalidArgumentError: Graph execution error:

Detected at node LogicalAnd_1 defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "C:\Users\Lenovo\AppData\Roaming\Python\Python311\site-packages\ipykernel_launcher.py", line 18, in <module>

  File "C:\Users\Lenovo\AppData\Roaming\Python\Python311\site-packages\traitlets\config\application.py", line 1075, in launch_instance

  File "C:\Users\Lenovo\AppData\Roaming\Python\Python311\site-packages\ipykernel\kernelapp.py", line 739, in start

  File "C:\Users\Lenovo\AppData\Roaming\Python\Python311\site-packages\tornado\platform\asyncio.py", line 205, in start

  File "c:\Users\Lenovo\AppData\Local\Programs\Python\Python311\Lib\asyncio\base_events.py", line 608, in run_forever

  File "c:\Users\Lenovo\AppData\Local\Programs\Python\Python311\Lib\asyncio\base_events.py", line 1936, in _run_once

  File "c:\Users\Lenovo\AppData\Local\Programs\Python\

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
import tensorflow as tf

# Load your data
df = pd.read_csv("../Dataset/optimal_fertilizer_train.csv")  # replace with your CSV if needed

# Step 1: Label encode the target
label_encoder = LabelEncoder()
df["Fertilizer Name"] = label_encoder.fit_transform(df["Fertilizer Name"])  # values 0–6

# Step 2: Define features and target
X = df.drop(columns=["id", "Fertilizer Name"])
y = df["Fertilizer Name"]

# Step 3: Train-test split
X_train_raw, X_test_raw, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Step 4: Column types
int_cols = ["Temparature", "Humidity", "Moisture", "Nitrogen", "Potassium", "Phosphorous"]
cat_cols = ["Soil Type", "Crop Type"]

# Step 5: Define preprocessing pipelines
num_pipeline = Pipeline([("scaler", StandardScaler())])
cat_pipeline = Pipeline([("onehot", OneHotEncoder(handle_unknown="ignore"))])

preprocessor = ColumnTransformer([
    ("num", num_pipeline, int_cols),
    ("cat", cat_pipeline, cat_cols),
])

# Step 6: Transform the data
X_train = preprocessor.fit_transform(X_train_raw)
X_test = preprocessor.transform(X_test_raw)

# Step 7: Convert to dense arrays if needed
if hasattr(X_train, "toarray"):
    X_train = X_train.toarray()
    X_test = X_test.toarray()

# Sanity checks
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)

# Step 8: Build model
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(X_train.shape[1],)),  # use shape, not input_shape
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(240, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(7, activation='softmax')  # 7 classes
])

model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

# Step 9: Setup callbacks
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_accuracy', patience=25, restore_best_weights=True
)

lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_accuracy', factor=0.3, patience=10, min_lr=1e-6
)

checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath='keras/ann_best_model.keras',
    monitor='val_accuracy',
    mode='max',
    save_best_only=True,
    verbose=1
)

# Step 10: Fit model
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=10,
    batch_size=32,
    callbacks=[early_stopping, lr_scheduler, checkpoint]
)


X_train shape: (562500, 22)
X_test shape: (187500, 22)
y_train shape: (562500,)
Epoch 1/10
12785/17579 ━━━━━━━━━━━━━━━━━━━━ 10s 2ms/step - accuracy: 0.1505 - loss: 1.9848

KeyboardInterrupt: 